In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import pandas as pd

In [39]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/anry/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
#https://medium.com/swlh/multi-label-text-classification-with-scikit-learn-and-tensorflow-257f9ee30536
def decontract(sentence):
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def removePunctuation(sentence): 
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)
    sentence = sentence.strip()
    sentence = sentence.replace("\n"," ")
    return sentence

def removeNumber(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    stop_words = set(stopwords.words('english'))
    sentence = sentence.split(' ')
    filtered_sentence = [w for w in sentence if not w.lower() in stop_words]
    return " ".join(filtered_sentence)
    

def stemming(sentence):
    stemmer = SnowballStemmer("english")
    stemmedSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemmedSentence += stem
        stemmedSentence += " "
    stemmedSentence = stemmedSentence.strip()
    return stemmedSentence

In [112]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train,y_train = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

maxlen = max(X_train.apply(lambda x:len(x)))+1

In [53]:
# X_train = X_train.apply(lambda x: decontract(x))
# X_train = X_train.apply(lambda x: removePunctuation(x))
# X_train = X_train.apply(lambda x: removeNumber(x))
# X_train = X_train.apply(lambda x: removeStopWords(x))
# X_train = X_train.apply(lambda x: stemming(x))

In [54]:
X_train

0                 know feel head get stuffier stuffier fun
1                          mum sound humbl sweet thing ask
2        star trek awesom kirk hot spock cool fun cool ...
3        amaz thing keep onlin convers one place unfort...
4        need water paper towel coke store feel sick go...
                               ...                        
26795                                excel never know lmao
26796                          new dress look sort horribl
26797    archetyp thing comm class studi archetyp popul...
26798                                   jealous pour right
26799                               road test book r scare
Name: content, Length: 26741, dtype: object

In [113]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, lower=True)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=maxlen)

In [76]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [117]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, LSTM, Input,Embedding,Dense
from tensorflow.keras.models import Model

max_words =10000
# maxlen = max(X_train.apply(lambda x:len(x)))+1

deep_inputs = Input(shape=(maxlen,))

embedding_layer = Embedding(max_words,120, trainable=True)(deep_inputs)# weights=[embedding_matrix],


LSTM_Layer_1 = LSTM(120,activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,dropout=0.1,return_sequences=True)(embedding_layer)

LSTM_Layer_2 = LSTM(120,activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True,dropout=0.1)(LSTM_Layer_1)


dense_layer_1 = Dense(13, activation='sigmoid')(LSTM_Layer_2)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=[  tf.keras.metrics.TruePositives(name='tp'),
  tf.keras.metrics.FalsePositives(name='fp'),
  tf.keras.metrics.TrueNegatives(name='tn'),
  tf.keras.metrics.FalseNegatives(name='fn'), 
  tf.keras.metrics.Precision(name='precision'),
  tf.keras.metrics.Recall(name='recall'),
  tf.keras.metrics.CategoricalAccuracy(name='acc'),
  tf.keras.metrics.AUC(name='auc')])

In [118]:
model.fit(X_train,pd.get_dummies(y_train),validation_split=0.2,batch_size=16,epochs = 4,workers = 4)

Epoch 1/4
1337/1337 [==============================] - 564s 405ms/step - loss: 0.2388 - tp: 14.0000 - fp: 132.0000 - tn: 256572.0000 - fn: 21378.0000 - precision: 0.0959 - recall: 6.5445e-04 - acc: 0.2145 - auc: 0.7667 - val_loss: 0.2285 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 64188.0000 - val_fn: 5349.0000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_acc: 0.2539 - val_auc: 0.7957
Epoch 2/4
1337/1337 [==============================] - 466s 349ms/step - loss: 0.2209 - tp: 184.0000 - fp: 168.0000 - tn: 256536.0000 - fn: 21208.0000 - precision: 0.5227 - recall: 0.0086 - acc: 0.3079 - auc: 0.8156 - val_loss: 0.2198 - val_tp: 79.0000 - val_fp: 51.0000 - val_tn: 64137.0000 - val_fn: 5270.0000 - val_precision: 0.6077 - val_recall: 0.0148 - val_acc: 0.3287 - val_auc: 0.8183
Epoch 3/4
1337/1337 [==============================] - 501s 374ms/step - loss: 0.2071 - tp: 1749.0000 - fp: 1254.0000 - tn: 255450.0000 - fn: 19643.0000 - precision: 0.5824 - recall: 0.0818 - acc: 

In [119]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = np.argmax(model.predict(X_train),axis = 1)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        74
           1       0.00      0.00      0.00       120
           2       0.00      0.00      0.00       546
           3       0.00      0.00      0.00       509
           4       0.00      0.00      0.00      1190
           5       0.42      0.56      0.48      3488
           6       0.00      0.00      0.00       886
           7       0.49      0.64      0.55      2573
           8       0.59      0.63      0.61      5744
           9       0.00      0.00      0.00      1022
          10       0.64      0.01      0.02      3459
          11       0.00      0.00      0.00      1465
          12       0.37      0.83      0.51      5665

    accuracy                           0.45     26741
   macro avg       0.19      0.20      0.17     26741
weighted avg       0.39      0.45      0.36     26741



/home/anry/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anry/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anry/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
y_train.value_counts(normalize=True)

8     0.214801
12    0.211847
5     0.130436
10    0.129352
7     0.096219
11    0.054785
4     0.044501
9     0.038218
6     0.033133
2     0.020418
3     0.019034
1     0.004487
0     0.002767
dtype: float64

In [103]:
np.argmax(y_pred,axis =1)

(26741,)

In [104]:
len(X_train)

26741